

# Proyecto

### Equipo:

- Felipe Jorquera Díaz

- \<Nombre de usuarios en Codalab\>

- \<Nombre del Equipo en Codalab\>

### Link de repositorio de GitHub: `https://github.com/felipejorqueradiaz`




## 1. Introducción

El objetivo de este proyecto consiste en ...

Los datos que proveen es un dataset con X ejemplos que describen una observación de ... 
Son N atributos y la variables objetivos son de tipo ... y ...

La primera tarea se evalua en base a la métrica ... ya que esta permite medir ...

Nuestra propuesta para resolver el problema consistió en modelo de clasificación basado en...

Nuestro modelo cumplió/no cumplió las expectativas...

---
## 2. Prepración y Análisis Exploratorio de Datos

In [1]:
import pandas as pd
import numpy as np
import umap
import plotly.express as px

#pip install pyarrow
#pip install seaborn
#pip install umap-learn

In [ ]:
with open('test.pickle', "rb") as fh:
    data_val = pickle.load(fh)
data_val['tipo'] = 'val' 

In [272]:
len(data_val)

656

In [274]:
data1 = pd.read_parquet('train_numerical_features.parquet')
data2 = pd.read_parquet('train_text_features.parquet')  
                           
data = data1.merge(data2, on = ['id', 'tagline', 'credits', 'title'])
data['tipo'] = 'traintest'
                           
data = pd.concat([data, data_val], axis=0)
                           
data.drop(['poster_path', 'backdrop_path', 'recommendations'], axis=1, inplace=True)

data = data[~ (
    (data['revenue']==0) |
    (data['release_date'].isna()) |
    (data['runtime'].isna()) |
    ((data['status'] != 'Released'))
                )]

data['release_date'] = pd.to_datetime(data['release_date'], format = '%Y-%m-%d')

data_cat = data.select_dtypes(include=[object])
data[data_cat.columns] = data_cat.fillna('')

data.loc[(data['vote_average'] <= 10) & (data['vote_average'] > 8), 'label'] = 'Very Positive'
data.loc[(data['vote_average'] <= 8) & (data['vote_average'] > 7), 'label'] = 'Positive'
data.loc[(data['vote_average'] <= 7) & (data['vote_average'] > 6), 'label'] = 'Mostly Positive'
data.loc[(data['vote_average'] <= 6) & (data['vote_average'] > 5), 'label'] = 'Mixed'
data.loc[(data['vote_average'] <= 5) , 'label'] = 'Negative'

data.drop(['vote_average', 'id', 'status'], axis=1, inplace=True)
data.rename(columns = {'revenue':'target'},
            inplace=True)

In [275]:
##  Código Preparación de Datos.

In [276]:
## Código EDA

```
Análisis del EDA.
```

---

## 3. Preprocesamiento, Holdout y Feature Engineering

In [277]:
from sklearn.preprocessing import StandardScaler

In [278]:
data['day'] = data['release_date'].dt.day
data['month'] = data['release_date'].dt.month
data['year'] = data['release_date'].dt.year
data['day_name'] = data['release_date'].dt.day_name()

data['en_language'] = np.where(data['original_language'] == 'en', 1, 0)

data['log_budget'] = np.log(data['budget'])
data['log_budget'].replace(np.inf, 0, inplace=True)
data['log_budget'].replace(-np.inf, 0, inplace=True)

C:\Users\Felipe\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [279]:
for col in ['genres', 'credits', 'production_companies']:
    values = [x.split(sep='-') for x in data[col]]
    len_values = [len(x) for x in values]
    data[f'len_{col}'] = len_values
    
    '''
    values = [i for x in values for i in x]
    val_unique, counts = np.unique(values, return_counts=True)
    series_count = pd.Series(counts, index = val_unique).sort_values(ascending = False).head(10)
    
    for value in series_count.index:
        data[f'{col}_{value}'] = pd.DataFrame(np.where(dataframe[col].str.contains(col), 1, 0))
    
    df_final = pd.concat(df.values(), axis=1)
    df_final.columns = df.keys()'''

In [280]:
popular_genres = ['Drama', 'Action', 'Comedy', 'Adventure', 'Thriller',
                 'Fantasy', 'Science Fiction', 'Family', 'Crime', 'Romance']
for pop in popular_genres:
    data[f'genre_{pop}'] = pd.DataFrame(np.where(data['genres'].str.contains(pop), 1, 0))
    
data['genre_Other'] = data.loc[:,data.columns[-len(popular_genres):]].sum(1)
data['genre_Popular'] = np.where(data['genre_Other']>0, 1, 0)
data['genre_Other'] = np.where(data['genre_Other']==0, 1, 0)

In [281]:
actores = [x.split(sep='-') for x in data['production_companies']]
values = [i for x in actores for i in x]
val_unique, counts = np.unique(values, return_counts=True)
series_count = pd.Series(counts, index = val_unique).sort_values(ascending = False).head(50)

counter2 = np.zeros(len(data))
for pop in series_count.index:
    counter2 = counter2 + np.where(data['production_companies'].str.contains(pop), 1, 0)
data['Popular_company'] = counter2

In [282]:
actores = [x.split(sep='-') for x in data['credits']]
values = [i for x in actores for i in x]
val_unique, counts = np.unique(values, return_counts=True)
series_count = pd.Series(counts, index = val_unique).sort_values(ascending = False).head(150)

counter2 = np.zeros(len(data))
for pop in series_count.index:
    counter2 = counter2 + np.where(data['credits'].str.contains(pop), 1, 0)
data['Popular_actor'] = counter2

In [283]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [284]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [285]:
#Seteamos el idioma en inglés
stop_words = stopwords.words('spanish')

class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

In [286]:
from sklearn.feature_extraction.text import CountVectorizer

In [287]:
data.drop(['release_date', 'original_language'], axis=1).head(2)

,title,budget,target,runtime,tagline,credits,genres,overview,production_companies,keywords,...,genre_Thriller,genre_Fantasy,genre_Science Fiction,genre_Family,genre_Crime,genre_Romance,genre_Other,genre_Popular,Popular_company,Popular_actor
0,Fantastic Beasts: The Secrets of Dumbledore,200000000.0,400000000.0,142.0,Return to the magic.,Jude Law-Eddie Redmayne-Mads Mikkelsen-Ezra Mi...,Fantasy-Adventure-Action,Professor Albus Dumbledore knows the powerful ...,Warner Bros. Pictures-Heyday Films,magic-curse-fantasy world-wizard-magical creat...,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1.0,1.0
1,Sonic the Hedgehog 2,110000000.0,393000000.0,122.0,Welcome to the next level.,James Marsden-Ben Schwartz-Tika Sumpter-Natash...,Action-Adventure-Family-Comedy,After settling in Green Hills Sonic is eager t...,Original Film-Blur Studio-Marza Animation Plan...,sequel-based on video game-hedgehog-live actio...,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1,2.0,2.0


In [288]:
## Código Holdout

In [289]:
## Código ColumnTransformer

In [290]:
## Código Feature Engineering (Opcional)

```
Comentarios
```

In [291]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [292]:
col_num = ['budget', 'runtime', 'log_budget']

In [293]:
preprocessing = ColumnTransformer(
    transformers = [
        ('StandardScaler', StandardScaler(), col_num),
        ('OneHotEncoder', OneHotEncoder(), ['day_name']),
        ('BagOfWords1', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                                              ), 'title'),
        ('BagOfWords2', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                                              ), 'overview')
    ])

In [294]:
data.columns

Index(['title', 'budget', 'target', 'runtime', 'tagline', 'credits', 'genres',
       'original_language', 'overview', 'production_companies', 'release_date',
       'keywords', 'tipo', 'label', 'day', 'month', 'year', 'day_name',
       'en_language', 'log_budget', 'len_genres', 'len_credits',
       'len_production_companies', 'genre_Drama', 'genre_Action',
       'genre_Comedy', 'genre_Adventure', 'genre_Thriller', 'genre_Fantasy',
       'genre_Science Fiction', 'genre_Family', 'genre_Crime', 'genre_Romance',
       'genre_Other', 'genre_Popular', 'Popular_company', 'Popular_actor'],
      dtype='object')

In [295]:
data_val = data[data['tipo'] == 'val'].drop('tipo', axis=1)
data = data[data['tipo'] != 'val'].drop('tipo', axis=1)

In [296]:
le = LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [297]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['target', 'credits', 'genres',
       'original_language', 'production_companies', 'release_date',
       'keywords', 'label'], axis=1),
    data['label'],
    shuffle = True,
    test_size = 0.2,
    random_state = 0
)

---

## 4. Clasificación

### 4.1 Dummy y Baseline

In [209]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [210]:
pipe1 = Pipeline(steps = [
    ('prepro', preprocessing),
    ('model', DecisionTreeClassifier(random_state=0))
])

In [211]:
#pip install xgboost

In [212]:
pipelines = {'DecisionTree': pipe1}

for name, model in pipelines.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores = classification_report(y_test, y_pred)
    print(f'Para el modelo {name} el performance es el siguiente:\n',scores,'\n\n')

Para el modelo DecisionTree el performance es el siguiente:
               precision    recall  f1-score   support

           0       0.31      0.25      0.28       272
           1       0.49      0.56      0.52       611
           2       0.00      0.00      0.00        31
           3       0.38      0.37      0.37       346
           4       0.12      0.06      0.08        31

    accuracy                           0.42      1291
   macro avg       0.26      0.25      0.25      1291
weighted avg       0.40      0.42      0.41      1291
 




In [213]:
pipe_grid = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', DecisionTreeClassifier())
])

In [216]:
params = [
    {
        'modelo': [LogisticRegression(random_state = 0)],
        'modelo__penalty': ['l1', 'l2'],
        'modelo__solver': ['liblinear'],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2), (1,3)]
    },{
        'modelo': [RandomForestClassifier(random_state=0)],
        'modelo__max_depth': [2,4,6,8,10],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2), (1,3)]
    },{
        'modelo': [xgb.XGBClassifier(random_state=0)],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2), (1,3)]
    }]

In [217]:
grid_model = GridSearchCV(pipe_grid,
                           param_grid=params,
                           verbose=10,
                            scoring = 'f1_macro',
                         cv=5)
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5; 1/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1)
[CV 1/5; 1/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1);, score=0.263 total time=   9.2s
[CV 2/5; 1/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1)
[CV 2/5; 1/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1);, score=0.241 total time=   9.0s
[CV 3/5; 1/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1)
[CV 3/5; 1/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords

[CV 1/5; 6/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3);, score=0.234 total time=  14.9s
[CV 2/5; 6/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 2/5; 6/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3);, score=0.242 total time=  14.1s
[CV 3/5; 6/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 3/5; 6/24] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3);, score=0.234 total time=  14.2s
[CV 4/5; 6/24] START modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 4/5

[CV 4/5; 11/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 2);, score=0.126 total time=  10.4s
[CV 5/5; 11/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 2)
[CV 5/5; 11/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 2);, score=0.126 total time=  10.7s
[CV 1/5; 12/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 1/5; 12/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 3);, score=0.126 total time=  11.0s
[CV 2/5; 12/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 2/5; 12/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=4, prepro__BagOfWords2__ngram_range=(1, 3)

[CV 3/5; 17/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 2);, score=0.126 total time=   9.9s
[CV 4/5; 17/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 2)
[CV 4/5; 17/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 2);, score=0.126 total time=  10.4s
[CV 5/5; 17/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 2)
[CV 5/5; 17/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 2);, score=0.126 total time=  10.5s
[CV 1/5; 18/24] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 3)
[CV 1/5; 18/24] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=8, prepro__BagOfWords2__ngram_range=(1, 3)

[CV 2/5; 22/24] END modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
              reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 1);, score=0.241 total time=  18.2s
[CV 3/5; 22/24] START modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
          

[CV 2/5; 23/24] END modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
              reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 2);, score=0.250 total time=  36.6s
[CV 3/5; 23/24] START modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
          

[CV 2/5; 24/24] END modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
              reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 3);, score=0.246 total time= 1.0min
[CV 3/5; 24/24] START modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
          

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepro',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         ['budget',
                                                                          'runtime',
                                                                          'log_budget']),
                                                                        ('OneHotEncoder',
                                                                         OneHotEncoder(),
                                                                         ['day_name']),
                                                                        ('BagOfWords1',
                                                                         CountVectorizer(ngram_range=(1,
                                                                                                      2),
                                                                                         tokenizer=<__main__.StemmerTokenizer object at 0x000001DB4DFCC040>),
                                                                         'title'),
                                                                        ('BagOfWords2',
                                                                         CountVect...
                                                   max_bin=None,
                                                   max_cat_to_onehot=None,
                                                   max_delta_step=None,
                                                   max_depth=None,
                                                   max_leaves=None,
                                                   min_child_weight=None,
                                                   missing=nan,
                                                   monotone_constraints=None,
                                                   n_estimators=100,
                                                   n_jobs=None,
                                                   num_parallel_tree=None,
                                                   predictor=None,
                                                   random_state=0,
                                                   reg_alpha=None,
                                                   reg_lambda=None, ...)],
                          'prepro__BagOfWords2__ngram_range': [(1, 1), (1, 2),
                                                               (1, 3)]}],
             scoring='f1_macro', verbose=10)

In [218]:
y_pred = grid_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.20      0.26       272
           1       0.51      0.72      0.60       611
           2       0.00      0.00      0.00        31
           3       0.46      0.38      0.42       346
           4       0.00      0.00      0.00        31

    accuracy                           0.48      1291
   macro avg       0.27      0.26      0.25      1291
weighted avg       0.44      0.48      0.45      1291



C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [219]:
import pickle

In [220]:
with open('grid_model.pickle', 'wb') as handle:
    pickle.dump(grid_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [268]:
y_val = grid_model.predict(data_val.drop(['target', 'credits', 'genres',
                                       'original_language', 'production_companies', 'release_date',
                                       'keywords', 'label'], axis=1))

In [305]:
yc_val = le.inverse_transform(y_val)

---

### 4.2 Búsqueda del mejor modelo de Clasificación


In [ ]:
### Código GridSearch

In [ ]:
### Código Predicción de datos de la competencia aquí

```
Justificación Aquí
```

---

## 5. Regresión

### 5.1 Dummy y Baseline

In [298]:
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    data.drop(['target', 'tagline', 'credits', 'genres',
       'original_language', 'production_companies', 'release_date',
       'keywords', 'label'], axis=1),
    data['target'],
    shuffle = True,
    test_size = 0.2,
    random_state = 0
)

In [299]:
Xr_train

,title,budget,runtime,overview,day,month,year,day_name,en_language,log_budget,...,genre_Thriller,genre_Fantasy,genre_Science Fiction,genre_Family,genre_Crime,genre_Romance,genre_Other,genre_Popular,Popular_company,Popular_actor
1134,The Wizard of Oz,2777000.0,102.0,Young Dorothy finds herself in a magical world...,15,8,1939,Tuesday,1,14.836882,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,3.0,0.0
7190,Sicko,9000000.0,123.0,Sicko is a Michael Moore documentary about the...,18,5,2007,Friday,1,16.012735,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1.0,0.0
8207,Yves Saint Laurent,12000000.0,106.0,A look at the life of French designer Yves Sai...,8,1,2014,Wednesday,0,16.300417,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0.0,2.0
1422,Epic,100000000.0,102.0,A teenager finds herself transported to a deep...,15,5,2013,Wednesday,1,18.420681,...,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1.0,1.0
6730,Chasing Amy,250000.0,114.0,Holden and Banky are comic book artists. Every...,4,4,1997,Friday,1,12.429216,...,0.0,0.0,0.0,0.0,0.0,1.0,0,1,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6751,The Farewell,250300.0,100.0,A headstrong Chinese-American woman returns to...,12,7,2019,Friday,1,12.430415,...,1.0,0.0,1.0,0.0,0.0,0.0,0,1,0.0,0.0
4327,Nutty Professor II: The Klumps,84000000.0,106.0,The hilarity begins when professor Sherman Klu...,27,7,2000,Thursday,1,18.246327,...,1.0,0.0,0.0,0.0,0.0,0.0,0,1,2.0,2.0
2140,The Dictator,65000000.0,83.0,The heroic story of a dictator who risks his l...,15,5,2012,Tuesday,1,17.989898,...,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1.0,3.0
3402,There Will Be Blood,25000000.0,158.0,Ruthless silver miner turned oil prospector Da...,26,12,2007,Wednesday,1,17.034386,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,2.0,2.0


In [300]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [301]:
pipe2 = Pipeline(steps = [
    ('prepro', preprocessing),
    ('model', LinearRegression())
])

In [302]:
pipelines = {'OLS': pipe2}

for name, model in pipelines.items():
    model.fit(Xr_train, yr_train)
    yr_pred = model.predict(Xr_test)
    
    scores = r2_score(yr_test, yr_pred)
    print(f'Para el modelo {name} el performance es el siguiente:\n',scores,'\n\n')

Para el modelo OLS el performance es el siguiente:
 0.503445240583869 




In [304]:
yr_val = model.predict(data_val.drop(['target', 'credits', 'genres',
                                       'original_language', 'production_companies', 'release_date',
                                       'keywords', 'label'], axis=1))

In [ ]:
## Código Dummy

In [ ]:
## Código Regresor

In [ ]:
## Código Comparación de métricas

```
Justificación
```

---

### 5.2 Búsqueda del mejor modelo de Regresión


In [ ]:
### Código GridSearch

In [ ]:
### Código Predicción de datos de la competencia aquí

---

## 6. Conclusiones

Conclusiones...



---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [308]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, y_pred_clf, y_pred_rgr): #clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    #y_pred_clf = clf_pipe.predict(predict_data)
    #y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [310]:
# Ejecutar función para generar el archivo de predicciones.
# perdict_data debe tener cargada los datos del text.pickle
# mientras que clf_pipe y rgr_pipe, son los pipeline de 
# clasificación y regresión respectivamente.
generateFiles(1, yc_val, yr_val)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>